In [19]:

import csv
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

# create a new Chrome browser instance
driver_path = '/path/to/chromedriver'
driver = webdriver.Chrome(driver_path)

# navigate to the webpage you want to scrape
url = 'https://www.theverge.com/'
driver.get(url)

# Find all the div tags with class name starting with "max-w-content-block"
divs = driver.find_elements(By.CSS_SELECTOR, 'div[class^="max-w-content-block"]')

# Write the titles, authors, and dates to a CSV file
now = datetime.datetime.now()
filename = f"{now.strftime('%d%m%Y')}_verge.csv"
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'URL', 'Title', 'author', 'date'])
    for i, div in enumerate(divs):
        h2_tag = div.find_element(By.TAG_NAME, 'h2')
        if h2_tag.find_element(By.TAG_NAME, 'a'):
            title = h2_tag.find_element(By.TAG_NAME, 'a').text
            url = h2_tag.find_element(By.TAG_NAME, 'a').get_attribute('href')
        else:
            title = 'Unknown'
            url = 'Unknown'
        author_div = div.find_elements(By.CSS_SELECTOR, 'a[href^="/authors/"]')
        if author_div:
            author_name = author_div[0].text
        else:
            author_name = 'Unknown'
        date_div = div.find_elements(By.CSS_SELECTOR, 'span[class^="text-gray-63"]')
        if date_div:
            date = date_div[0].text
        else:
            date = 'Unknown'
        writer.writerow([i+1, url, title, author_name, date])

# Close the browser window
driver.quit()


<ipython-input-19-30d4930cbd55>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [35]:
import csv
import sqlite3

# Establish a connection to the SQLite database
conn = sqlite3.connect('verge_art.db')
cursor = conn.cursor()

Create a table called "articles" with columns for id, url, title, author, and date
cursor.execute('''
    CREATE TABLE articles (
        id INTEGER PRIMARY KEY,
        url TEXT NOT NULL,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        date TEXT NOT NULL
   )
''')

# Read the data from the CSV file and insert it into the articles table
with open(f"{now.strftime('%d%m%Y')}_verge.csv", newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # skip the header row
    for row in reader:
        cursor.execute('INSERT INTO articles VALUES (?, ?, ?, ?, ?)', row)

# Commit the changes and close the connection
conn.commit()
conn.close()


SyntaxError: invalid syntax (<ipython-input-35-9fd7773444e9>, line 8)

In [20]:
# to insert into database and create unique id for them

import sqlite3
import csv

# Establish a connection to the SQLite database
conn = sqlite3.connect('verge_arti.db')

# Create a table called "articles" with columns for id, url, title, author, and date
cursor = conn.cursor()

# Create a table called "articles" with columns for id, url, title, author, and date
cursor.execute('''
    CREATE TABLE articles (
        id INTEGER PRIMARY KEY,
        url TEXT NOT NULL,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        date TEXT NOT NULL
    )
''')

# Read the data from the CSV file and insert it into the database
with open(f"{now.strftime('%d%m%Y')}_verge.csv", 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip header row
    for row in reader:
        cursor.execute('INSERT OR IGNORE INTO articles (url, title, author, date) VALUES (?, ?, ?, ?)', row[1:])

# Commit the changes and close the connection
conn.commit()
conn.close()


In [33]:
###will insert data and check if title is already there skip that row

import csv
import sqlite3

# Establish a connection to the SQLite database
conn = sqlite3.connect('verge_arti.db')
cursor = conn.cursor()

# Read the data from the CSV file and insert it into the articles table
with open(f"{now.strftime('%d%m%Y')}_verge.csv", newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # skip the header row
    for row in reader:
        # Check if the title already exists in the database
        cursor.execute('SELECT id FROM articles WHERE title = ?', (row[2],))
        result = cursor.fetchone()
        if result is not None:
            # Title already exists, skip this row
            continue
        # Title does not exist, insert the row
        cursor.execute('INSERT INTO articles (url, title, author, date) VALUES (?, ?, ?, ?)', (row[1], row[2], row[3], row[4]))

# Commit the changes and close the connection
conn.commit()
conn.close()
